In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
# https://www.kaggle.com/datasets/abdelghaniaaba/wildfire-prediction-dataset

In [ ]:
import os
from PIL import Image

root_dir = "/path/to/dataset"
min_size = 56

for subdir, dirs, files in os.walk(root_dir):
    for file in files:
        filepath = os.path.join(subdir, file)
        try:
            with Image.open(filepath) as im:
                if im.size[0]*im.size[1] < min_size:
                    os.remove(filepath)
                    print(f"{filepath} removed.")
        except:
            os.remove(filepath)
            print(f"{filepath} removed.")


In [10]:
import os
from PIL import Image

def is_image_truncated(filepath):
    try:
        with Image.open(filepath) as img:
            img.verify()
    except:
        return True
    return False

def clean_dataset(root_dir):
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            filepath = os.path.join(dirpath, filename)
            if is_image_truncated(filepath):
                print(f"Removing {filepath}")
                os.remove(filepath)

# root_dir = "path/to/your/dataset"
clean_dataset("C:/Users/Bhavani K/Desktop/M GANESHV12/M PES/M SEM 4/M hashcode/test/nowildfire")
clean_dataset("C:/Users/Bhavani K/Desktop/M GANESHV12/M PES/M SEM 4/M hashcode/test/wildfire")
clean_dataset("C:/Users/Bhavani K/Desktop/M GANESHV12/M PES/M SEM 4/M hashcode/valid/nowildfire")
clean_dataset("C:/Users/Bhavani K/Desktop/M GANESHV12/M PES/M SEM 4/M hashcode/valid/wildfire")
clean_dataset("C:/Users/Bhavani K/Desktop/M GANESHV12/M PES/M SEM 4/M hashcode/train/nowildfire")
clean_dataset("C:/Users/Bhavani K/Desktop/M GANESHV12/M PES/M SEM 4/M hashcode/train/wildfire")


In [12]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        'train/',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
        'valid/',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        'test/',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')


Found 30250 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.


In [13]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))


In [16]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy'])


In [7]:
history = model.fit(train_generator,
                    steps_per_epoch=50,
                    epochs=10,
                    validation_data=validation_generator,
                    validation_steps=50)


Epoch 1/10
50/50 [==============================] - 39s 779ms/step - loss: 0.2781 - accuracy: 0.8856 - val_loss: 0.2722 - val_accuracy: 0.8863
Epoch 2/10
50/50 [==============================] - 40s 805ms/step - loss: 0.2879 - accuracy: 0.8819 - val_loss: 0.2043 - val_accuracy: 0.9144
Epoch 3/10
50/50 [==============================] - 39s 789ms/step - loss: 0.2674 - accuracy: 0.8821 - val_loss: 0.2905 - val_accuracy: 0.8919
Epoch 4/10
50/50 [==============================] - 39s 781ms/step - loss: 0.2461 - accuracy: 0.9062 - val_loss: 0.2130 - val_accuracy: 0.9194
Epoch 5/10
50/50 [==============================] - 39s 774ms/step - loss: 0.2257 - accuracy: 0.9125 - val_loss: 0.1897 - val_accuracy: 0.9219
Epoch 6/10
50/50 [==============================] - 40s 794ms/step - loss: 0.2425 - accuracy: 0.9094 - val_loss: 0.3128 - val_accuracy: 0.8781
Epoch 7/10
50/50 [==============================] - 39s 776ms/step - loss: 0.2258 - accuracy: 0.9162 - val_loss: 0.1959 - val_accuracy: 0.9287

In [1]:
# test_loss, test_acc = model.evaluate(test_generator)
# print('Test accuracy:', test_acc)


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = "C:/Users/Bhavani K/Desktop/M GANESHV12/M PES/M SEM 4/M hashcode/test/wildfire/-60.6867,50.26079.jpg"
# "C:\Users\Bhavani K\Desktop\M GANESHV12\M PES\M SEM 4\M hashcode\test\wildfire\-60.6867,50.26079.jpg"
img = image.load_img(img_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.

prediction = model.predict(img_array)
if prediction < 0.5:
    print("The image is not a wildfire.")
else:
    print("The image is a wildfire.")


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

def predict_image(image_path, model):
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.

    prediction = model.predict(img_array)
    if prediction < 0.5:
        return "The image is not a wildfire."
    else:
        return "The image is a wildfire."

    
image_path = 'path/to/new/image.jpg'
prediction = predict_image(image_path, model)
print(prediction)
